In [13]:
import numpy as np
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder
# We start by getting access to the drive

In [14]:
import pandas as pd

train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

Vamos a ver qué tal se ha cargado

## Eliminando atributos no correctos

Vamos a eliminar del conjunto de entrenamiento los atributos 'Artista' y
'Titulo' porque no se puede aprender a partir de ellos.

También quitamos el atributo a predecir para evitar usarlo de entrada.

In [15]:
train_target = train_df.Category
targetencoder = LabelEncoder()
train_target = targetencoder.fit_transform(train_target)
train_df.drop(['Artista', 'Titulo', 'Category'], axis=1, inplace=True)
train_df.columns

Index(['bailable', 'energia', 'clave', 'hablado', 'positivismo',
       'duración en min/ms', 'acustico', 'vivacidad', 'popularidad', 'tempo',
       'modo', 'volumen', 'ritmo', 'instrumental'],
      dtype='object')

## Viendo nulos

Ahora voy a ver los nulos y procesarlos. En general hay que procesarlos mejor
pero aquí únicamente voy a reemplazarlos por las medias.

In [16]:
train_df.replace({'P1.0': 1.0, 'P9.0': 9.0, 'P10.0':10.0, 'P11.0':11.0, 'P2.0': 2.0, 'P6.0': 6.0, 'P4.0': 4.0,
       'P7.0': 7.0, 'P5.0': 5.0 , 'P8.0': 8.0, 'P3.0' : 3.0, 'menor': 0, 'mayor': 1}, inplace=True)
test_df.replace({'P1.0': 1.0, 'P9.0': 9.0, 'P10.0':10.0, 'P11.0':11.0, 'P2.0': 2.0, 'P6.0': 6.0, 'P4.0': 4.0,
       'P7.0': 7.0, 'P5.0': 5.0 , 'P8.0': 8.0, 'P3.0' : 3.0, 'menor': 0, 'mayor': 1}, inplace=True)

In [17]:
ids = test_df['id']
test_df.drop(columns="id", inplace=True)


In [18]:
import sklearn
from sklearn.impute import SimpleImputer

imputer1 = SimpleImputer(strategy='mean')
train_df = imputer1.fit_transform(train_df)
test_df = imputer1.transform(test_df)

In [20]:
from sklearn.ensemble import HistGradientBoostingClassifier

clf = HistGradientBoostingClassifier(random_state=0)
clf.fit(train_df, train_target)
clf.score(train_df, train_target)

0.7320969646454122

In [18]:
from sklearn.model_selection import GridSearchCV
clf = HistGradientBoostingClassifier(random_state=0)
parameters = [{"learning_rate": np.linspace(0.023, 0.029, 30)}]
search = GridSearchCV(clf, parameters, cv=5, scoring='accuracy', n_jobs=-1)
search.fit(train_df, train_target)

GridSearchCV(cv=5, estimator=HistGradientBoostingClassifier(random_state=0),
             n_jobs=-1,
             param_grid=[{'learning_rate': array([0.023     , 0.0232069 , 0.02341379, 0.02362069, 0.02382759,
       0.02403448, 0.02424138, 0.02444828, 0.02465517, 0.02486207,
       0.02506897, 0.02527586, 0.02548276, 0.02568966, 0.02589655,
       0.02610345, 0.02631034, 0.02651724, 0.02672414, 0.02693103,
       0.02713793, 0.02734483, 0.02755172, 0.02775862, 0.02796552,
       0.02817241, 0.02837931, 0.02858621, 0.0287931 , 0.029     ])}],
             scoring='accuracy')

In [19]:
search.best_params_

{'learning_rate': 0.024241379310344827}

In [20]:
search.cv_results_

{'mean_fit_time': array([7.22449336, 7.21609435, 7.23409204, 7.03712335, 7.39970713,
        7.36401062, 7.27031207, 7.40806427, 7.3205977 , 7.34720407,
        7.18591866, 7.31514621, 7.40748329, 7.02765946, 7.12349687,
        7.13440857, 7.1159101 , 7.02956972, 7.24282579, 7.26534982,
        7.07447572, 7.21699982, 7.12294736, 7.2485569 , 7.24281087,
        7.25344076, 7.14855227, 7.41578932, 6.84848852, 4.83340425]),
 'std_fit_time': array([0.08074426, 0.11458395, 0.09439912, 0.04299742, 0.09024338,
        0.09401165, 0.12369474, 0.14853589, 0.12506406, 0.22415534,
        0.06525815, 0.17672595, 0.20333553, 0.0630782 , 0.06885447,
        0.10278916, 0.1423447 , 0.03688331, 0.11841587, 0.26361739,
        0.03099322, 0.18602277, 0.06180905, 0.23558467, 0.14950636,
        0.27442828, 0.07763096, 0.3006274 , 0.79255554, 0.12982309]),
 'mean_score_time': array([0.25400128, 0.26240015, 0.25820236, 0.25259972, 0.27966881,
        0.28478251, 0.27098331, 0.2630116 , 0.2627984 , 0.25

In [21]:
clf = HistGradientBoostingClassifier(random_state=0, learning_rate=0.024241379310344827)
clf.fit(train_df, train_target)
clf.score(train_df, train_target)

0.6686809752031673

In [31]:
from sklearn.model_selection import GridSearchCV
clf = HistGradientBoostingClassifier(random_state=0, learning_rate=0.024241379310344827)
parameters = [{"l2_regularization": np.linspace(0.001, 0.005, 50)}]
search = GridSearchCV(clf, parameters, cv=5, scoring='accuracy', n_jobs=-1)
search.fit(train_df, train_target)

GridSearchCV(cv=5,
             estimator=HistGradientBoostingClassifier(learning_rate=0.024241379310344827,
                                                      random_state=0),
             n_jobs=-1,
             param_grid=[{'l2_regularization': array([0.001     , 0.00108163, 0.00116327, 0.0012449 , 0.00132653,
       0.00140816, 0.0014898 , 0.00157143, 0.00165306, 0.00173469,
       0.00181633, 0.00189796, 0.00197959, 0.00206122, 0.00214286,
       0.00222449, 0.00230612, 0.00238776...102,
       0.00263265, 0.00271429, 0.00279592, 0.00287755, 0.00295918,
       0.00304082, 0.00312245, 0.00320408, 0.00328571, 0.00336735,
       0.00344898, 0.00353061, 0.00361224, 0.00369388, 0.00377551,
       0.00385714, 0.00393878, 0.00402041, 0.00410204, 0.00418367,
       0.00426531, 0.00434694, 0.00442857, 0.0045102 , 0.00459184,
       0.00467347, 0.0047551 , 0.00483673, 0.00491837, 0.005     ])}],
             scoring='accuracy')

In [32]:
search.best_params_

{'l2_regularization': 0.0020612244897959186}

In [33]:
search.cv_results_

{'mean_fit_time': array([6.91100607, 6.97140336, 6.93040152, 6.86837988, 6.90462346,
        6.99797659, 6.94495368, 6.928967  , 6.97856674, 6.98399606,
        6.88574433, 6.96871181, 7.02440681, 6.91484985, 6.94828439,
        7.04924431, 6.87153549, 6.88907471, 6.95617661, 6.92080278,
        6.89002514, 6.89062161, 6.8830584 , 6.90328436, 6.90390401,
        6.89370255, 6.90900073, 6.96421714, 6.93908477, 6.85523491,
        6.90614152, 6.97462182, 6.84064808, 6.84982462, 6.97328367,
        6.88175597, 6.88036537, 6.98211689, 7.56301775, 7.10763922,
        7.34441786, 7.18073254, 7.0391088 , 7.22137661, 7.22364426,
        6.95222321, 6.9640698 , 7.09939933, 5.8917932 , 5.48431334]),
 'std_fit_time': array([0.05477875, 0.03044722, 0.08926851, 0.06637441, 0.01059476,
        0.05369826, 0.08694621, 0.03735051, 0.07344254, 0.07794358,
        0.04562607, 0.11544833, 0.14124453, 0.04464159, 0.07860919,
        0.07356577, 0.04388553, 0.09741252, 0.10227077, 0.07201582,
        0.043

In [34]:
clf = HistGradientBoostingClassifier(random_state=0, learning_rate=0.024241379310344827, l2_regularization=0.0020612244897959186)
clf.fit(train_df, train_target)
clf.score(train_df, train_target)

0.6663193720914079

In [38]:
from sklearn.model_selection import GridSearchCV
clf = HistGradientBoostingClassifier(random_state=0, learning_rate=0.024241379310344827, l2_regularization=0.0020612244897959186)
parameters = [{"max_leaf_nodes": np.arange(10, 21, 1)}]
search = GridSearchCV(clf, parameters, cv=5, scoring='accuracy', n_jobs=-1)
search.fit(train_df, train_target)

GridSearchCV(cv=5,
             estimator=HistGradientBoostingClassifier(l2_regularization=0.0020612244897959186,
                                                      learning_rate=0.024241379310344827,
                                                      random_state=0),
             n_jobs=-1,
             param_grid=[{'max_leaf_nodes': array([10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20])}],
             scoring='accuracy')

In [40]:
search.best_params_

{'max_leaf_nodes': 19}

In [41]:
search.cv_results_

{'mean_fit_time': array([3.34229512, 3.60644383, 3.91914091, 3.90649858, 4.09134555,
        4.34751701, 4.41912074, 4.49162135, 4.75608945, 4.4874896 ,
        3.89425859]),
 'std_fit_time': array([0.05832565, 0.11812074, 0.18838694, 0.06663443, 0.0770053 ,
        0.05548149, 0.14263481, 0.01892168, 0.07643545, 0.45947685,
        0.10306808]),
 'mean_score_time': array([0.21269403, 0.23129921, 0.22583833, 0.2309031 , 0.22848053,
        0.22931895, 0.23275971, 0.23975139, 0.23932443, 0.2134006 ,
        0.18379908]),
 'std_score_time': array([0.00353642, 0.01575955, 0.00741019, 0.0091961 , 0.00291619,
        0.00532741, 0.00356938, 0.00612743, 0.01326569, 0.01575555,
        0.00132598]),
 'param_max_leaf_nodes': masked_array(data=[10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20],
              mask=[False, False, False, False, False, False, False, False,
                    False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'max_leaf_nodes': 10},
  {

In [42]:
clf = HistGradientBoostingClassifier(random_state=0, learning_rate=0.024241379310344827, l2_regularization=0.0020612244897959186, max_leaf_nodes=19)
clf.fit(train_df, train_target)
clf.score(train_df, train_target)

0.6271445440022226

In [43]:
from sklearn.model_selection import GridSearchCV
clf = HistGradientBoostingClassifier(random_state=0, learning_rate=0.024241379310344827, l2_regularization=0.0020612244897959186, max_leaf_nodes=19)
parameters = [{"min_samples_leaf": np.arange(10, 40, 1)}]
search = GridSearchCV(clf, parameters, cv=5, scoring='accuracy', n_jobs=-1)
search.fit(train_df, train_target)

GridSearchCV(cv=5,
             estimator=HistGradientBoostingClassifier(l2_regularization=0.0020612244897959186,
                                                      learning_rate=0.024241379310344827,
                                                      max_leaf_nodes=19,
                                                      random_state=0),
             n_jobs=-1,
             param_grid=[{'min_samples_leaf': array([10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26,
       27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39])}],
             scoring='accuracy')

In [44]:
search.best_params_

{'min_samples_leaf': 21}

In [45]:
search.cv_results_

{'mean_fit_time': array([4.99290919, 5.00690856, 4.98410621, 4.930059  , 4.93999519,
        4.96744113, 4.94147615, 4.93261499, 4.96236105, 4.91895452,
        4.92166977, 4.90188408, 4.93132677, 4.84416037, 4.90003724,
        4.87702732, 4.85697756, 4.86339426, 4.89162707, 4.80337944,
        4.82189274, 4.83091927, 4.83886752, 4.77462668, 4.81069307,
        4.86946931, 4.92538476, 4.9227879 , 4.65519223, 3.32608633]),
 'std_fit_time': array([0.03652634, 0.04588815, 0.03949461, 0.02985148, 0.04041482,
        0.04644201, 0.02830101, 0.05964099, 0.09156167, 0.08409443,
        0.05751879, 0.07913171, 0.06989568, 0.01947947, 0.05319775,
        0.04065151, 0.03036942, 0.01941148, 0.0815466 , 0.01772451,
        0.03903656, 0.06072623, 0.05327099, 0.03442096, 0.05237226,
        0.02456974, 0.03015128, 0.05092682, 0.62787579, 0.03549667]),
 'mean_score_time': array([0.22320271, 0.23624492, 0.22966633, 0.22909837, 0.22668891,
        0.23405991, 0.2301753 , 0.23825407, 0.23662925, 0.23

In [46]:
from sklearn.model_selection import GridSearchCV
clf = HistGradientBoostingClassifier(random_state=0, learning_rate=0.024241379310344827, l2_regularization=0.0020612244897959186, max_leaf_nodes=19, min_samples_leaf=21)
parameters = [{"max_depth": np.arange(10, 40, 1)}]
search = GridSearchCV(clf, parameters, cv=5, scoring='accuracy', n_jobs=-1)
search.fit(train_df, train_target)

GridSearchCV(cv=5,
             estimator=HistGradientBoostingClassifier(l2_regularization=0.0020612244897959186,
                                                      learning_rate=0.024241379310344827,
                                                      max_leaf_nodes=19,
                                                      min_samples_leaf=21,
                                                      random_state=0),
             n_jobs=-1,
             param_grid=[{'max_depth': array([10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26,
       27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39])}],
             scoring='accuracy')

In [47]:
search.best_params_

{'max_depth': 17}

In [48]:
search.cv_results_

{'mean_fit_time': array([4.92091107, 5.07095609, 5.12519827, 4.87006054, 5.07656898,
        5.19996862, 5.02676849, 4.89416628, 4.91142468, 4.88901629,
        4.86420999, 4.85488229, 4.88974609, 4.87119775, 4.85379949,
        4.90420022, 4.92861276, 4.98804774, 5.09664698, 4.9084332 ,
        4.87412891, 4.88272877, 4.86474419, 4.82681584, 4.90681725,
        4.94697409, 4.91750669, 4.97150302, 4.70250788, 3.38877521]),
 'std_fit_time': array([0.11223912, 0.13294935, 0.08090993, 0.05006354, 0.11909953,
        0.19445398, 0.2134665 , 0.0172403 , 0.03988522, 0.05301203,
        0.03182349, 0.04435582, 0.05104415, 0.04617124, 0.04257237,
        0.04687512, 0.08612985, 0.05436344, 0.03550447, 0.11826942,
        0.07406683, 0.03396862, 0.04114443, 0.02966432, 0.027532  ,
        0.0439522 , 0.01834607, 0.0935111 , 0.61695818, 0.10039144]),
 'mean_score_time': array([0.2324894 , 0.2372448 , 0.24260216, 0.23387461, 0.24819932,
        0.25199962, 0.25100079, 0.23859344, 0.23599944, 0.24

In [21]:
clf = HistGradientBoostingClassifier(random_state=0, learning_rate=0.024241379310344827, l2_regularization=0.0020612244897959186, max_leaf_nodes=19, min_samples_leaf=21, max_depth=30)
clf.fit(train_df, train_target)
clf.score(train_df, train_target)

0.6275612974925332

# FULL GRID

In [23]:
from sklearn.model_selection import GridSearchCV
clf = HistGradientBoostingClassifier(random_state=0, l2_regularization=0.0020612244897959186)
parameters = [{"learning_rate": np.linspace(0.023, 0.029, 5),  "max_leaf_nodes": np.arange(10, 22, 2), "min_samples_leaf": np.arange(10, 40, 6), "max_depth": np.arange(10, 40, 6)}]
search = GridSearchCV(clf, parameters, cv=5, scoring='accuracy', n_jobs=-1)
search.fit(train_df, train_target)

GridSearchCV(cv=5,
             estimator=HistGradientBoostingClassifier(l2_regularization=0.0020612244897959186,
                                                      random_state=0),
             n_jobs=-1,
             param_grid=[{'learning_rate': array([0.023 , 0.0245, 0.026 , 0.0275, 0.029 ]),
                          'max_depth': array([10, 16, 22, 28, 34]),
                          'max_leaf_nodes': array([10, 12, 14, 16, 18, 20]),
                          'min_samples_leaf': array([10, 16, 22, 28, 34])}],
             scoring='accuracy')

In [27]:
from sklearn.model_selection import cross_val_score
clf = HistGradientBoostingClassifier(random_state=0, l2_regularization=0.0020612244897959186, learning_rate=0.026000000000000002,
  max_depth=22, max_leaf_nodes=20, min_samples_leaf=28)
scores = cross_val_score(clf, train_df, train_target, cv=5)
scores

array([0.54166667, 0.54444444, 0.54289684, 0.53421327, 0.55991664])

In [31]:
from sklearn.model_selection import GridSearchCV
clf = HistGradientBoostingClassifier(random_state=0, l2_regularization=0.0020612244897959186, learning_rate=0.026000000000000002,
  max_depth=22, max_leaf_nodes=20, min_samples_leaf=28)
parameters = [{"random_state": np.arange(50, 200, 1)}]
search = GridSearchCV(clf, parameters, cv=5, scoring='accuracy', n_jobs=-1)
search.fit(train_df, train_target)

GridSearchCV(cv=5,
             estimator=HistGradientBoostingClassifier(l2_regularization=0.0020612244897959186,
                                                      learning_rate=0.026000000000000002,
                                                      max_depth=22,
                                                      max_leaf_nodes=20,
                                                      min_samples_leaf=28,
                                                      random_state=0),
             n_jobs=-1,
             param_grid=[{'random_state': array([ 50,  51,  52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,
        63,  64,  65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,
        76,  77,  78,  79,  80,  81,  82,  83,  84...
       115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127,
       128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140,
       141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153,
       154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166,
       167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179,
       180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192,
       193, 194, 195, 196, 197, 198, 199])}],
             scoring='accuracy')

In [32]:
search.best_params_, search.best_score_

({'random_state': 184}, 0.5461556626529273)

In [28]:
scores.mean()

0.5446275713017636

In [24]:
search.best_params_, search.best_score_

({'learning_rate': 0.026000000000000002,
  'max_depth': 22,
  'max_leaf_nodes': 20,
  'min_samples_leaf': 28},
 0.5446275713017636)

In [34]:
from collections import Counter
print(Counter(train_target))

Counter({10: 3984, 1: 2071, 0: 2012, 8: 1463, 9: 1152, 6: 1103, 3: 1036, 4: 501, 7: 442, 2: 321, 5: 312})


In [35]:
clf = HistGradientBoostingClassifier(random_state=8, l2_regularization=0.0020612244897959186, learning_rate=0.026000000000000002,
  max_depth=22, max_leaf_nodes=20, min_samples_leaf=28)
clf.fit(train_df, train_target)
clf.score(train_df, train_target)

0.630895325415017

Y finalmente aplico el modelo entrenado (ignorando id):

In [36]:
predictions = clf.predict(test_df)
predictions = targetencoder.inverse_transform(predictions)
print(predictions)

['Acustico/Folk' 'Rock' 'Blues' ... 'Rock' 'Rock' 'Rock']


Ahora que tengo las predicciones voy a crear el fichero

In [37]:
submission = pd.DataFrame({'id': ids, 'Expected': predictions})

Vemos que tenemos el tamaño adecuado:

In [38]:
submission.shape

(3599, 2)

Creamos el fichero del modelo:

In [39]:
submission.to_csv("IGV2a2023.csv", index=False)